# Exercise 3 - Technical Comparison of an SQL and Graph Database

For this exercise you are going to set up a MySQL database and a Neo4J database. You will initialize the databases with data of an artificial social network. That network consists of persons, i.e., users of a platform such as LinkedIn, and endorsements, i.e., the acknowledgment of another person. 

The data is given to you in CSV files. You can see small examples here, where one CSV file contains the data for the graph nodes (entities) and the other CSV file contains the data for the edges:

https://github.com/HelgeCPH/db_course_nosql/blob/master/social_network/social_network_nodes_small.csv
https://github.com/HelgeCPH/db_course_nosql/blob/master/social_network/social_network_edges_small.csv

**OBS:** The two files above are only small examples to understand how the data is structured. For your exercise you will use the **complete** dataset containing 500.000 nodes and some millions of edges:

https://github.com/HelgeCPH/db_course_nosql/blob/master/social_network/archive_graph.tar.gz.


Precisely, your task is:

  1. Setup a MySQL and a Neo4j database respectively.
  2. Import the data from the social network (endorsement graph https://github.com/HelgeCPH/db_course_nosql/blob/master/social_network/archive_graph.tar.gz) into a Neo4J database and into a MySQL database repectively.
  3. Construct queries in SQL and in Cypher, which find

     1. all persons that a person endorses, i.e., endorsements of depth one.
     2. all persons that are endorsed by endorsed persons of a person, i.e., endorsements of depth two.
     3. endorsements of depth three.
     4. endorsements of depth four.
     5. endorsements of depth five.

  4. Write a Java program that executes the above queries for twenty random nodes against the two respective databases.
  5. Extend your Java program, so that it measures the execution times of each query, i.e., you run a benchmark for the two databases.
  6. You collect your measurement results and present them with an evaluation of your experiment in a Markdown file on your Github account. That is, you hand in this assignment via Github, with one hand-in per group.
    * Present the execution time of each query each of the 20 random nodes/persons per database.
    * Present the average and the median runtime of each of the queries per database.
  7. Push your solution, source, code, and presentation of the results to a Github repository per group *and* inform Helge about that.
  8. The *hand-in* time is latest 2. April 2017 at 24o'clock.


Hints:

  1. In case you do not want to install a MySQL database and a Neo4J database locally, you can make use of a VM, which is configured by the Vagrantfile at:
  The VM runs a MySQL and Neo4J database accessible on the specified ports.
  
  **OBS** The VM in is standard configuration is low on RAM. To receive meaningful results that do not meassure only swapping times you have to increase VM RAM and tell the Neo4J configuration about it. Both can be done in the Vagrant file:
    * Destroy your current VM.
    * Now edit your Vagrant file.
      * Increase RAM: change `vb.memory = "2048"` to `vb.memory = "8192"`. Of course you have to have so much RAM on your machine.
      * Increase the initial heap space and the max heap size in the `neo4j.conf`, for example to:
    
        ```
        echo "dbms.memory.heap.initial_size=4096m" >> /etc/neo4j/neo4j.conf
        echo "dbms.memory.heap.max_size=7168m" >> /etc/neo4j/neo4j.conf
        ```
        
    * Save your modified Vagrantfile and start the machine up again with `vagrant up`
    * Alternatively, you can install the database engines locally, see for example: https://neo4j.com/download/community-edition/
 
  2. In case you are using the VM, the data for the social graph in CSV files is already downloaded and unzipped to the path `/var/lib/neo4j/import`. One file for the persons (`social_network_nodes.csv`) and another one for the endorsement relations (`social_network_edges.csv`).

  **OBS** Importing the above files will take quite some time. In particular importing the many millions of relation ships takes ca. 10 minutes on the MySQL database and up to 20 minutes on Neo4J in the VM.

  3. In MySQL you can load a CSV file as in the following
  
```sql
LOAD DATA LOCAL INFILE '/var/lib/neo4j/import/social_network_nodes.csv'
INTO TABLE yourdb.t_user
FIELDS TERMINATED BY ',' OPTIONALLY ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 LINES
(id,name,job,birthday)
;
```
  
  4. Present your measurement results in an understandable way. For example in a table similar to the following:
  
  ```
  			         MySQL			    Neo4j		
                   average	 median	average median
Depth one:	   0.5       0.4       2.1     1.4
Depth two:	   1.8       17.0      2.5     2.1
Depth three:     57.5      17.0      11.5    7.3
Depth four:	  1716.1    517.9     208.9   80.5
Depth five:	  204874.4  70712.5   4647.8  1549.0
```

This exercise is inspired and similar to chapter 1 "A case for a Neo4J database" of the book "Neo4J in Action" https://www.manning.com/books/neo4j-in-action). You can read the first chapter as PDF: https://manning-content.s3.amazonaws.com/download/5/392c9aa-4c64-4c6d-a072-fcf6b73d4ef1/Neo4jinAction_CH01.pdf


# Accessing MySQL from Java

  * Create a Maven project. In NetBeans `New Project -> Maven -> Java Application`
  * Add a dependency to the MySQL driver to your project configuration (`pom.xml`) 
  
```xml
    <dependencies>
        <dependency>
            <groupId>mysql</groupId>
            <artifactId>mysql-connector-java</artifactId>
            <version>5.1.40</version>
        </dependency> 
        
        <!-- ...any other dependencies -->
    </dependencies>
```

  * Create a Java Class `ConnectionTest.java` and type in the following code:
  

```java
package dk.cphbusiness.db.neo4j.intro;

import java.sql.Connection;
import java.sql.DriverManager;
import java.sql.ResultSet;
import java.sql.SQLException;
import java.sql.Statement;

/**
 *
 * @author Helge
 */
public class ConnectionTest {

    public static void main(String[] args) {
        Connection con = null;
        Statement st = null;
        ResultSet rs = null;

        String url = "jdbc:mysql://localhost:3306/your_db";
        String user = "root";
        String password = "pwd";
                
        String query = "SELECT * FROM your_db.t_user";
        
        try {
            
            con = DriverManager.getConnection(url, user, password);
            
            st = con.createStatement();
            rs = st.executeQuery(query);

            if (rs.next()) {                
                System.out.println(rs.getString(1));
            }

        } catch (SQLException ex) {
            System.err.println(ex.getMessage());
            System.err.println(ex);
        } finally { 
            try {
                if (rs != null) {
                    rs.close();
                }
                if (st != null) {
                    st.close();
                }
                if (con != null) {
                    con.close();
                }
            } catch (SQLException ex) {
                System.err.println(ex.getMessage());
                System.err.println(ex);
            }
        }
    }
}
```

This program should print a record for each person in your MySQL database `your_db` in which you created a table `t_user`.